# Presentation on Word2Vec
Glenn Abastillas | 24 March, 2020

This notebook goes over an example implementation of Word2Vec and some existing packages that perform Word2Vec training.

Contents
  1.  Preliminary Steps
      * [Load Packages](#load_packages)
      * [Preprocess Data](#preprocess_data)
      * [Quick Background](#quick_background)
  2. [Implementation from Scratch](#implementation_from_scratch)
      * Word2Vec Flavors: Continuous Back of Words (CBOW) / Skip Grams (SG)
      * Training
      * Retrieving the trained matrix
  3. Using an Existing Package

__At each step, we will also cover other packages that can be used to acheive the same thing (e.g., Countvectorizer)__
  
---

### Load Packages <a id="load_packages"></a>
First we import packages and clean the data.

In [1]:
import numpy as np
import spacy
import tqdm
from string import punctuation
from nltk.corpus import gutenberg, stopwords

We will use data from the `gutenberg` corpus and normalize the input data.

In [2]:
sents = gutenberg.sents('melville-moby_dick.txt')

`[insert navigation here]`

---
### Preprocess Data <a id="preprocess_data"></a>
##### Normalize Vocabulary

To improve processing and richness of our lexical items, we normalize our language data. 

Normalizing data can involve a variety of tasks depending on the final application of our language model. These tasks including making all words the same case, removing punctuation, and removing **<a id="stopword" style="text-decoration: none; cursor: help;" title="Words that contribute little semantic information to a text">stopwords</a>**.

For this presentation, we will use **<a id="token" style="text-decoration: none; cursor: help;" title="Combinations of characters separated by spaces (e.g., words, numbers)">tokens</a>** that are not punctuation nor stopwords.

Let's quickly define some functions we will use to pare our text data down.

In [3]:
stopwords_ = stopwords.words('english')

def is_stopword(token):
    ''' Check if a specified token is a stopword. 
    
        Parameters
        ----------
            token (str) : token to check
        
        Returns
        -------
            (boolean) True if token is a stopword, else False
        
        Notes
        -----
            Input that are not strings will return as False
    '''
    try:
        return token.lower() in stopwords_
    except:
        return False

def is_valid_token(token):
    ''' Check if token is valid, i.e., not a stopword or punctuation
    
        Parameters
        ----------
            token (str) : token to check
        
        Returns
        -------
            (boolean) True if token is a stopword, else False
        
        Notes
        -----
            Input that are not strings will return as False
    '''
    try:
        return token.isalnum() & ~is_stopword(token)
    except:
        return False

##### First Normalization Step
Next, we create our `raw_text` data using the functions we just defined.

In [11]:
%%time
normalized_sents = [[word.lower() for word in sent if is_valid_token(word)] for sent in sents]

Wall time: 2.45 s


With our `normalized_text`, we can create a `dict` to convert the strings into numbers for faster processing down the line.

In [6]:
flattened_text = [word for sent in normalized_sents for word in sent]

VOCAB, INDEX = np.unique(flattened_text, return_index=True)

We now define a `to_index` function to convert strings to integers for faster processing.

In [8]:
def to_index(token, reference=VOCAB):
    ''' Convert an input token into an integer according to a specified reference
        
        Parameters
        ----------
            token (str) : token to convert to an integer
            reference (list, array) : Reference array with unique vocabulary
        
        Returns
        -------
            An integer representing the input token's position in the reference object
    
        Notes
        -----
            Function assumes input token is already in lower case
    '''
    return np.argwhere(reference == token)[0][0]

##### Second Normalization Step

Using the conversion function defined above, we can convert our `normalized_sents` into `data`, which contains only integers that will be used in our Word2Vec example.

In [29]:
%%time
data = ([to_index(word) for word in sent] for sent in normalized_sents)

Wall time: 2.96 ms


`[insert navigation here]`

---
### Preprocessing Text

Use this reference for later [Dimensions greater than 300 have diminishing returns](https://www.aclweb.org/anthology/D14-1162/)

### Quick Background <a id="quick_background"></a>
#### Background Into Word2Vec

* What is it?
  - Quick definition (implementation of theoretical matrix bit)
  - What is does to text.
  - What the output is.
  
* Why do we need it?
  - Many uses in AI.
  - Usage in NLP
  - Pros
  - Cons
 
* What cool things can it do?
  - Condense text into a lightweight matrix
  - Provide semantic abilities
  - Enable data to have algebraic properties
 
* What are competing models?
  - Other models to represent text
  - GloVe
  - Other vectorization models

`[insert navigation here]`

---
## Implementation from Scratch <a id="implementation_from_scratch"></a>

For things example, we will create a Word2Vec language model using the data we preprocessed above. 

In this section, we will develop a **<a id="cbow" style="text-decoration: none; cursor: help;" title="Continuous Bag of Words">CBOW</a>** flavored Word2Vec model.

We will:
  * Create CBOW windows
  * Create preliminary <a id='one-hot' style='text-decoration: none; cursor: help;' title='A vector that is comprised of zeros and ones indicating absence or presence of a value'>one-hot vectors</a>


###### Parameters <a id='parameters'></a>
First we define some hyperparameters that we use for training.

In [20]:
parameters = {'window_size' : 2, 'dimensions' : 100, 'learning_rate' : 0.02, 'epochs' : 500}

This table quickly describes what each parameter does.

Parameters | Data Type | Description
--- | :-: | :--
`window_size` | `int` | The number of target tokens before and after a central token to include
`dimensions` | `int` | The number of dimensions in hidden layer. Dimensions greater than 300 have diminishing returns `[cite]`.
`learning_rate` | `float` | How quickly our model will correct itself
`epochs` | `int` | The number of rounds the model is trained

###### Creating the Training Data <a id="creating_the_training_data"></a>

**Windowing** : We will generate loose <a id='one-hot' style='text-decoration: none; cursor: help;' title='A vector that is comprised of zeros and ones indicating absence or presence of a value'>one-hot vectors</a> that will serve as input and target data when training our model.

First we filter our data to ensure we have sufficient data to window.

In [30]:
%%time
data = [sent for sent in data if len(sent) >= parameters['window_size'] + 1]

Wall time: 36.7 s


Now, we generate our one hot vectors using the `VOCAB` as a model for our vector.

Let's define a few functions to help use generate these data.

In [72]:
from collections import namedtuple

In [78]:
Vector = namedtuple('Vector', 'this was that'.split())
Vector(1, 2, 3)

Vector(this=1, was=2, that=3)

In [69]:
def one_hot_vector(token, size=VOCAB.size):
    ''' Convert an input token into an integer according to a specified reference
        
        Parameters
        ----------
            token (str) : token to convert to a one-hot array
            size (int) : length of one-hot array to return
        
        Returns
        -------
            (np.array) A one-hot vector
    '''
    vector = np.zeros(VOCAB.size)
    vector[token] = 1
    return vector

def process_sentence(sent, window_size=parameters['window_size'] + 1):
    ''' Return a dictionary with token keys and contexts as values
        
        Parameters
        ----------
            sent (list) : sentence to convert into one-hot vectors 
                          and group into target and context
            window_size (int) : Window size of CBOW
        
        Returns
        -------
            (dict) object with tokens as keys and their corresponding
                   one-hot vector targets and context in the following format:
            
            >> { token : [[target, [context-1, context-2, ...]], ...]}
    '''
    processed_sentence = {}
    context = []
    
    for i, token in enumerate(sent):
        before = sent[max(i-window_size, 0):i]
        after = sent[i:i+window_size]

        for context_token in before + after:
            context.append(one_hot_vector(context_token))
        
        target = one_hot_vector(token)
        processed_sentence[token] = [target, context]
        context = []
    return processed_sentence

In [70]:
process_sentence(data[3])

{5183: [array([0., 0., 0., ..., 0., 0., 0.]),
  [array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.])]],
 4684: [array([0., 0., 0., ..., 0., 0., 0.]),
  [array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.])]],
 10185: [array([0., 0., 0., ..., 0., 0., 0.]),
  [array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.])]],
 8669: [array([0., 0., 0., ..., 0., 0., 0.]),
  [array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.]),
   array([0., 0., 0., ..., 0., 0., 0.])]],
 6541: [array([0., 0., 0., ..., 0., 0., 0.]),
  [array([0., 0., 0., .

In [40]:
vector = np.zeros(VOCAB.size)

for datum in data[:1]:
    for word in datum:
        print(one_hot_vector(word), word)

[0. 0. 0. ... 0. 0. 0.] 9521
[0. 0. 0. ... 0. 0. 0.] 4196
[0. 0. 0. ... 0. 0. 0.] 7041
[0. 0. 0. ... 0. 0. 0.] 9312
[0. 0. 0. ... 0. 0. 0.] 82


## Using an Existing Package

There are existing implementations that already exist that allow you to use Word2Vec technology out of the box. In this section, we look at the Python package `gensim` with an API implementing Word2Vec.

Contents
  * SpaCy
  * gensim
 
 
### SpaCy

In [ ]:
nlp = spacy.load('en_core_web_sm')